In [1]:
### Execute ligand target analysis for lagged ligands 

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/stark_stemi_R_Env_4_1//lib/R/library"



Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
   

In [3]:
source('MS4_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


###############################################
# Preqrequisites Configurations & Parameters

In [4]:
data_path ='../data/current'

In [5]:
result_path = '../results/current'

In [6]:
data_path

[1] "../data/current"

In [7]:
### Version of the pre-processed data to load

In [8]:
name = 'V_FINAL'  
# of dataset to load, prepared and combined data of E1 - skip E2 step 

In [9]:
filter_samples = 'acs'
#filter_samples = 'acs_w_o_infection_ccs'
#filter_samples = 'no_ccs_ccs'

# Load Data 

## Sample Data

In [10]:
## sample meta and clinical data

In [11]:
sample_data = read.csv(paste0(result_path, '/00_Data_Overview/Merged_Sample_Meta_Data.csv'))

## Prepared combined data

In [12]:
### Select and load pre-processed and combined data

In [13]:
path = paste0(result_path, '/E-Analysis/Combined_Data_',name,'.csv')
print(path)
data_long = read.csv(path)
print(file.info(path)$mtime)

[1] "../results/current/E-Analysis/Combined_Data_V_FINAL.csv"
[1] "2024-01-03 14:22:52 CET"


In [14]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [15]:
nrow(data_long)

[1] 1557939

In [16]:
length(unique(data_long$variable))

[1] 13289

## Ligand Receptor Data

In [17]:
## Load ligand receptor data from nichenet database

In [18]:
lr_network = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds")) # ligand receptor interactions

In [19]:
head(lr_network,2)

from,to,source,database
<chr>,<chr>,<chr>,<chr>
CXCL1,CXCR2,kegg_cytokines,kegg
CXCL2,CXCR2,kegg_cytokines,kegg


In [20]:
## Summarise to have only ligand or receptor assignment not double

In [21]:
ligand_receptor_interactions = lr_network %>% group_by(from,to) %>% count()

In [22]:
colnames(ligand_receptor_interactions) = c('Ligand', 'Receptor', 'amount_sources')

In [23]:
head(ligand_receptor_interactions,2)

Ligand,Receptor,amount_sources
<chr>,<chr>,<int>
A2M,LRP1,1
A2M,TNFRSF14,1


In [24]:
### Aggregate to contain only unique ligand-receptor interactions across different data sources

In [25]:
ligand_amounts = ligand_receptor_interactions %>% group_by(Ligand) %>% summarise(amount_l = sum(amount_sources))

In [26]:
receptor_amounts = ligand_receptor_interactions %>% group_by(Receptor) %>% summarise(amount_r = sum(amount_sources))

In [27]:
l_r_merged = merge(ligand_amounts, receptor_amounts, by.x = 'Ligand', by.y = 'Receptor')

In [28]:
ligand = l_r_merged$Ligand[l_r_merged$amount_l >= l_r_merged$amount_r]

In [29]:
receptor = l_r_merged$Ligand[l_r_merged$amount_l < l_r_merged$amount_r]

In [30]:
nrow(l_r_merged)

[1] 115

In [31]:
head(l_r_merged,2)

,Ligand,amount_l,amount_r
,<chr>,<int>,<int>
1,ALCAM,3,2
2,AMIGO1,2,1


In [32]:
length(ligand)

[1] 84

In [33]:
head(ligand)

[1] "ALCAM"  "AMIGO1" "CADM1"  "CADM3"  "CADM4"  "CALM1"

In [34]:
length(receptor)

[1] 31

In [35]:
### Filter out duplicate assignments and decide whether it is a receptor or ligand

In [36]:
head(ligand_receptor_interactions,2)

Ligand,Receptor,amount_sources
<chr>,<chr>,<int>
A2M,LRP1,1
A2M,TNFRSF14,1


In [37]:
ligand_receptor_interactions = ligand_receptor_interactions[!ligand_receptor_interactions$Receptor %in% ligand,]

In [38]:
ligand_receptor_interactions = ligand_receptor_interactions[!ligand_receptor_interactions$Ligand %in% receptor,]

In [39]:
nrow(ligand_receptor_interactions)

[1] 11639

In [40]:
## Generate dataset specifying ligand receptor variables

In [41]:
ligand_receptor_vars = rbind(data.frame(gene = ligand_receptor_interactions$Ligand, interaction_type = 'Ligand'), data.frame(gene = ligand_receptor_interactions$Receptor, interaction_type = 'Receptor'))

In [42]:
ligand_receptor_vars = unique(ligand_receptor_vars)

In [43]:
head(ligand_receptor_vars,2)

,gene,interaction_type
,<chr>,<chr>
1,A2M,Ligand
4,AANAT,Ligand


In [44]:
nrow(ligand_receptor_vars)

[1] 1414

In [45]:
# remove empty spaces
ligand_receptor_vars$gene = str_replace(ligand_receptor_vars$gene, ' ', '')

## Available ligand receptor potential + transcription factor datasets from  nichenet

In [46]:
## Weighted networks

In [47]:
weighted_networks = readRDS(url("https://zenodo.org/record/3260758/files/weighted_networks.rds")) # integrated weighted ligand-signaling and gene regulatory network

In [48]:
## Prepared influential matrices

In [49]:
ligand_target_matrix = readRDS(url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds"))  # gene-gene regulatory scores
# gives the regulatory potential of a target gene based on the ligand-tf & tf- target-gene links and weights of the signaling and gr network (multiply PPR of each ligand -tf  pair with the weights of the gr network)

In [50]:
### Generate data in long format

In [51]:
ligand_target_matrix_long = melt(ligand_target_matrix)

In [52]:
colnames(ligand_target_matrix_long) = c('target', 'ligand', 'regulatory_potential')

In [53]:
head(ligand_target_matrix_long,2)

,target,ligand,regulatory_potential
,<fct>,<fct>,<dbl>
1,A1BG,CXCL1,0.0003534343
2,A1BG-AS1,CXCL1,0.0001650894


In [54]:
length(unique(ligand_target_matrix_long$ligand))

[1] 688

## Cell Gene Expression Data (percentage of cells expressing gene)

In [55]:
## Load information about how many cells express a gene generated in C0 Script

In [56]:
path = paste0(result_path, '/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values.csv')
cell_perc_cluster = read.csv( path) # cluster alternative
print(file.info(path)$mtime)
#cell_perc_cluster = read.csv( paste0(result_path, '/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values_cell_type_Scanorama.csv')) # cell-type alternative

[1] "2024-01-03 14:10:30 CET"


In [57]:
### Filter on relevant cell types (clusters with low amount of cells per sample; similar to MOFA analysis)

In [58]:
cell_perc_cluster = cell_perc_cluster[!(cell_perc_cluster$cluster %in% c('16_Plasma Blast', '14_Other', '17_Progenitor', '15_Plasma Blast', '18_Megakaryocytes' ,'Plasma Blast', 'Other', 'Progenitor', 'Plasma Blast', 'Megakaryocytes' )),]

In [59]:
unique(cell_perc_cluster$cluster)

[1] "8_B-cell"                   "9_Monocytes - CD16_FCGR3A" 
 [3] "5_T-cell-CD4"               "0_T-cell-CD4"              
 [5] "1_T-cell-CD8"               "6_Monocytes - CD14"        
 [7] "2_T-cell-CD4"               "4_Monocytes - CD14"        
 [9] "11_T-cell-CD4"              "3_NK"                      
[11] "7_Monocytes - CD14"         "13_Dendritic"              
[13] "10_B-cell"                  "12_Monocytes - CD16_FCGR3A"

In [60]:
head(cell_perc_cluster,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,8_B-cell
2,AL627309.4,0.05406136,4,AL627309.4,8_B-cell


# Potential Pre-Processing / Data Adjustments/ Data Preparations

## Prepare list of cytokines in data

In [61]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [62]:
cytokines = unique(data_long$variable[data_long$type == 'cytokine'])

In [63]:
head(cytokines)

[1] "BCA1__CXCL13"    "CTACK__CCL27"    "EGF__EGF"        "ENA78__CXCL5"   
[5] "Eotaxin__CCL11"  "Eotaxin2__CCL24"

In [64]:
cytokines = str_replace(cytokines, '.*__', '') # extract gene code from cytokines

In [65]:
head(cytokines)

[1] "CXCL13" "CCL27"  "EGF"    "CXCL5"  "CCL11"  "CCL24"

In [66]:
nrow(ligand_receptor_interactions[ligand_receptor_interactions$Ligand %in% cytokines,])

[1] 1236

## Adjustments to data_long

### Filter samples (optional)

In [67]:
## filter out some samples if specified in the beginning

In [68]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [69]:
unique(sample_data$classification)

[1] "vollstaendiger_ausschluss" "ccs"                      
[3] "koronarsklerose"           "acs_subacute"             
[5] "acs_w_infection"           "acs_w_o_infection"

In [70]:
if(filter_samples == 'all'){
    filter = unique(sample_data$sample_id)
    }

if(filter_samples == 'acs'){
    filter = unique(sample_data$sample_id[sample_data$classification %in% c('acs_w_infection', 'acs_w_o_infection', 'acs_subacute')])
    }


if(filter_samples == 'acs_w_o_infection_ccs'){
    filter =unique(sample_data$sample_id[sample_data$classification %in% c('ccs', 'acs_w_o_infection')])
    
    }

if(filter_samples == 'no_ccs_ccs'){
    filter =unique(sample_data$sample_id[sample_data$classification %in% c('vollstaendiger_ausschluss','ccs','koronarsklerose')])
    
    }

In [71]:
length(filter)

[1] 112

In [72]:
#filter

In [73]:
nrow(data_long)

[1] 1557939

In [74]:
data_long = data_long[data_long$sample_id %in% filter,]

In [75]:
nrow(data_long)

[1] 1128736

In [76]:
unique(data_long$sample_id)  # only acs samples with timepoints

[1] "m1.1"  "m10.2" "m10.3" "m11.2" "m11.3" "m11.4" "m12.1" "m12.2" "m13.1"
 [10] "m13.2" "m13.3" "m13.4" "m14.1" "m14.2" "m15.1" "m15.2" "m15.3" "m15.4"
 [19] "m16.1" "m16.2" "m17.1" "m18.1" "m18.2" "m18.3" "m19.1" "m19.2" "m19.3"
 [28] "m2.1"  "m2.2"  "m2.3"  "m2.4"  "m20.3" "m20.4" "m21.1" "m21.2" "m21.3"
 [37] "m21.4" "m22.1" "m22.2" "m22.3" "m22.4" "m23.1" "m23.2" "m23.3" "m23.4"
 [46] "m24.2" "m24.3" "m24.4" "m25.1" "m25.2" "m25.3" "m26.1" "m26.2" "m26.3"
 [55] "m26.4" "m27.1" "m27.2" "m27.3" "m28.2" "m28.3" "m28.4" "m3.2"  "m3.3" 
 [64] "m3.4"  "m4.1"  "m4.2"  "m4.3"  "m4.4"  "m5.1"  "m5.3"  "m5.4"  "m6.1" 
 [73] "m6.2"  "m6.3"  "m6.4"  "m7.1"  "m7.2"  "m7.3"  "m7.4"  "m8.1"  "m8.2" 
 [82] "m8.3"  "m8.4"  "m9.1"  "m9.2"  "m9.3"  "m11.1" "m10.1" "m20.1" "m24.1"
 [91] "m5.2"  "m20.2" "m14.3" "m14.4" "m1.2"  "m1.3"  "m1.4"  "m10.4" "m12.3"
[100] "m12.4" "m16.3" "m16.4" "m17.2" "m17.3" "m17.4" "m18.4" "m19.4" "m25.4"
[109] "m27.4" "m28.1" "m3.1"  "m9.4"

### Filter on expression

In [77]:
## Filter out features with low expression (as in E2 script)

In [78]:
data_long$expression = TRUE

In [79]:
data_long$expression[data_long$value == 0] = FALSE

In [80]:
expression_filter = data_long %>% group_by(type, variable) %>% summarise(perc_expression = sum(expression)  )

`summarise()` has grouped output by 'type'. You can override using the
`.groups` argument.


In [81]:
expression_filter$perc_expression = expression_filter$perc_expression / length(unique(data_long$sample_id))

In [82]:
nrow(expression_filter)

[1] 13289

In [83]:
filtered_out = expression_filter[expression_filter$perc_expression <= 0.2,]

In [84]:
expression_filter = expression_filter[expression_filter$perc_expression > 0.2,]

In [85]:
nrow(expression_filter)

[1] 13282

In [86]:
filtered_out # features that are filtered out because of too low expressionv

type,variable,perc_expression
<chr>,<chr>,<dbl>
cytokine,IL16__IL16,0.07142857
cytokine,IL21__IL21,0.11607143
cytokine,IL23__IL23,0.11607143
cytokine,IL28A__IL28,0.08928571
cytokine,IL33__IL33,0.13392857
cytokine,IL4__IL4,0.19642857
cytokine,TSLP__TSLP,0.15178571


In [87]:
# filter the data
data_long = merge(data_long, expression_filter[,c('type', 'variable')], by.x = c('type', 'variable'), by.y = c('type', 'variable'))   # filter the data

In [88]:
unique(data_long$type)

[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
[5] "single_cell"

In [89]:
nrow(data_long)

[1] 1128092

In [90]:
data_long$ident = paste0(data_long$type, '_0_', data_long$variable)

### Extract gene format

In [91]:
## Extract gene codes from variables

In [92]:
data_long$gene = str_replace(data_long$variable, '.*__', '')

In [93]:
data_long$gene[data_long$type == 'proteomics'] = str_replace(data_long$gene[data_long$type == 'proteomics'], '_.*', '')

### Data filtered for ligands and receptors

In [94]:
### Filter dataset on ligand and receptor variables

In [95]:
data_long_lig_rec = merge(data_long, ligand_receptor_vars)

In [96]:
# overview amount of ligands and receptors measured per dimension 
unique(data_long_lig_rec[,c('type', 'interaction_type', 'gene')]) %>% group_by(type, interaction_type) %>% count()

type,interaction_type,n
<chr>,<chr>,<int>
cytokine,Ligand,60
neutrophil,Ligand,24
neutrophil,Receptor,43
proteomics,Ligand,53
proteomics,Receptor,19
single_cell,Ligand,53
single_cell,Receptor,75


In [97]:
## generate a variable defining type: cell-type cluster or dimension

In [98]:
data_long_lig_rec$cluster = data_long_lig_rec$type

In [99]:
data_long_lig_rec$cluster[data_long_lig_rec$type == 'single_cell'] = str_replace(data_long_lig_rec$variable[data_long_lig_rec$type == 'single_cell'] , '__.*', '')

In [100]:
unique(data_long_lig_rec$cluster)

[1] "proteomics"                  "neutrophil"                 
 [3] "X6_Monocytes...CD14"         "X4_Monocytes...CD14"        
 [5] "X7_Monocytes...CD14"         "X12_Monocytes...CD16_FCGR3A"
 [7] "X3_NK"                       "X0_T.cell.CD4"              
 [9] "X10_B.cell"                  "X13_Dendritic"              
[11] "X5_T.cell.CD4"               "X9_Monocytes...CD16_FCGR3A" 
[13] "X11_T.cell.CD4"              "X1_T.cell.CD8"              
[15] "X2_T.cell.CD4"               "X8_B.cell"                  
[17] "cytokine"

In [101]:
unique(data_long_lig_rec$type)

[1] "proteomics"  "neutrophil"  "single_cell" "cytokine"

In [102]:
head(data_long_lig_rec,2)

,gene,type,variable,X,sample_id,value,config,expression,ident,interaction_type,cluster
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,A2M,proteomics,A2M_P01023,1439457,m25.1,8.823516,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,proteomics
2,A2M,proteomics,A2M_P01023,1439527,m4.3,9.797027,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,proteomics


### Add information about percentage of cells expressing gene

In [103]:
## add to the ligand receptor dataset

In [104]:
cell_perc_cluster$cluster = make.names(cell_perc_cluster$cluster)

In [105]:
nrow(data_long_lig_rec)

[1] 69043

In [106]:
data_long_lig_rec$X = NULL

In [107]:
head(cell_perc_cluster,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,X8_B.cell
2,AL627309.4,0.05406136,4,AL627309.4,X8_B.cell


In [108]:
head(data_long_lig_rec,2)

,gene,type,variable,sample_id,value,config,expression,ident,interaction_type,cluster
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,A2M,proteomics,A2M_P01023,m25.1,8.823516,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,proteomics
2,A2M,proteomics,A2M_P01023,m4.3,9.797027,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,proteomics


In [109]:
unique(data_long_lig_rec$cluster)

[1] "proteomics"                  "neutrophil"                 
 [3] "X6_Monocytes...CD14"         "X4_Monocytes...CD14"        
 [5] "X7_Monocytes...CD14"         "X12_Monocytes...CD16_FCGR3A"
 [7] "X3_NK"                       "X0_T.cell.CD4"              
 [9] "X10_B.cell"                  "X13_Dendritic"              
[11] "X5_T.cell.CD4"               "X9_Monocytes...CD16_FCGR3A" 
[13] "X11_T.cell.CD4"              "X1_T.cell.CD8"              
[15] "X2_T.cell.CD4"               "X8_B.cell"                  
[17] "cytokine"

In [110]:
unique(cell_perc_cluster$cluster)

[1] "X8_B.cell"                   "X9_Monocytes...CD16_FCGR3A" 
 [3] "X5_T.cell.CD4"               "X0_T.cell.CD4"              
 [5] "X1_T.cell.CD8"               "X6_Monocytes...CD14"        
 [7] "X2_T.cell.CD4"               "X4_Monocytes...CD14"        
 [9] "X11_T.cell.CD4"              "X3_NK"                      
[11] "X7_Monocytes...CD14"         "X13_Dendritic"              
[13] "X10_B.cell"                  "X12_Monocytes...CD16_FCGR3A"

In [111]:
## merge the data
data_long_lig_rec = merge(data_long_lig_rec, cell_perc_cluster, all.x = TRUE) # also keep cytokine, proteomics dimension where no cell expression is available

In [112]:
head(data_long_lig_rec,2)

,gene,cluster,type,variable,sample_id,value,config,expression,ident,interaction_type,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<int>
1,A2M,proteomics,proteomics,A2M_P01023,m25.1,8.823516,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,NA,NA,NA
2,A2M,proteomics,proteomics,A2M_P01023,m4.3,9.797027,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,NA,NA,NA


In [113]:
# Set dummy mean expression for proteomic/ cytokine data etc. where no mean cell expression values are available
data_long_lig_rec$perc_cells_expressing_gene[is.na(data_long_lig_rec$perc_cells_expressing_gene)] = mean(data_long_lig_rec$perc_cells_expressing_gene, na.rm = TRUE)

In [114]:
unique(data_long_lig_rec$type)

[1] "proteomics"  "neutrophil"  "single_cell" "cytokine"

###  Aggregate expression across samples (mean expression etc.)

In [115]:
## for ligand- receptor data

In [116]:
### Calculate variance and mean expression per cell-type across samples for single cell data

analyze_expr_cluster = data_long_lig_rec %>% group_by(type, interaction_type, gene, cluster, perc_cells_expressing_gene, total_amount_cells_expressing_gene, ident) %>% summarise(mean_expr = mean(value, na.rm = TRUE), var_expression = var(value, na.rm = TRUE))

`summarise()` has grouped output by 'type', 'interaction_type', 'gene',
'cluster', 'perc_cells_expressing_gene', 'total_amount_cells_expressing_gene'.
You can override using the `.groups` argument.


In [117]:
### Generate cell type columns
analyze_expr_cluster$cell_type = str_replace(analyze_expr_cluster$cluster, 'X.*_', '')

In [118]:
head(analyze_expr_cluster,2)

type,interaction_type,gene,cluster,perc_cells_expressing_gene,total_amount_cells_expressing_gene,ident,mean_expr,var_expression,cell_type
<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<chr>
cytokine,Ligand,CCL1,cytokine,69.10381,NA,cytokine_0_I309__CCL1,1.381370,0.2566531,cytokine
cytokine,Ligand,CCL11,cytokine,69.10381,NA,cytokine_0_Eotaxin__CCL11,7.376516,2.8469700,cytokine


In [119]:
unique(analyze_expr_cluster$cell_type)

[1] "cytokine"         "neutrophil"       "proteomics"       "Monocytes...CD14"
 [5] "T.cell.CD4"       "B.cell"           "FCGR3A"           "Dendritic"       
 [9] "T.cell.CD8"       "NK"

# Ligand Receptor Data

## Define ligand and receptors

In [120]:
head(data_long_lig_rec,2)

,gene,cluster,type,variable,sample_id,value,config,expression,ident,interaction_type,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<int>
1,A2M,proteomics,proteomics,A2M_P01023,m25.1,8.823516,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,NA,69.10381,NA
2,A2M,proteomics,proteomics,A2M_P01023,m4.3,9.797027,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,proteomics_0_A2M_P01023,Ligand,NA,69.10381,NA


In [121]:
### Define Ligands and Receptors

In [122]:
ligands = unique(data_long_lig_rec$ident[data_long_lig_rec$interaction_type == 'Ligand'])

In [123]:
length(ligands)

[1] 444

In [124]:
receptors = unique(data_long_lig_rec$ident[data_long_lig_rec$interaction_type == 'Receptor'])

In [125]:
length(receptors)

[1] 367

## Add nichenet interaction potential

In [126]:
head(ligand_receptor_interactions,2)

Ligand,Receptor,amount_sources
<chr>,<chr>,<int>
A2M,LRP1,1
A2M,TNFRSF14,1


In [127]:
head(weighted_networks$lr_sig,2)

from,to,weight
<chr>,<chr>,<dbl>
A1BG,ABCC6,0.4216439
A1BG,ACE2,0.1007411


In [128]:
# merge the two datasets
ligand_receptor_interactions =merge( ligand_receptor_interactions, weighted_networks$lr_sig, by.x = c('Ligand', 'Receptor'), by.y = c('from','to'))

In [129]:
nrow(ligand_receptor_interactions)

[1] 11639

In [130]:
head(ligand_receptor_interactions,2)

,Ligand,Receptor,amount_sources,weight
,<chr>,<chr>,<int>,<dbl>
1,A2M,LRP1,1,0.4678977
2,A2M,TNFRSF14,1,0.6990671


In [131]:
#### Add expression data

In [132]:
unique(analyze_expr_cluster$type)

[1] "cytokine"    "neutrophil"  "proteomics"  "single_cell"

In [133]:
head(analyze_expr_cluster,2)

type,interaction_type,gene,cluster,perc_cells_expressing_gene,total_amount_cells_expressing_gene,ident,mean_expr,var_expression,cell_type
<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<chr>
cytokine,Ligand,CCL1,cytokine,69.10381,NA,cytokine_0_I309__CCL1,1.381370,0.2566531,cytokine
cytokine,Ligand,CCL11,cytokine,69.10381,NA,cytokine_0_Eotaxin__CCL11,7.376516,2.8469700,cytokine


In [134]:
analyze_expr_cluster$variable = paste0(analyze_expr_cluster$cluster, '_0_', analyze_expr_cluster$gene)

In [135]:
nrow(ligand_receptor_interactions)

[1] 11639

In [136]:
#### Add data for ligands
data  = merge(ligand_receptor_interactions, analyze_expr_cluster[analyze_expr_cluster$interaction_type == 'Ligand',], by.x = c('Ligand'), by.y = c('gene'),  suffixes = c("","_L"))

In [137]:
#### Add data for Receptors
data  = merge(data, analyze_expr_cluster[analyze_expr_cluster$interaction_type == 'Receptor',], by.x = c('Receptor'), by.y = c('gene'),  suffixes = c("","_R"))

In [138]:
head(data,2)

,Receptor,Ligand,amount_sources,weight,type,interaction_type,cluster,perc_cells_expressing_gene,total_amount_cells_expressing_gene,ident,⋯,type_R,interaction_type_R,cluster_R,perc_cells_expressing_gene_R,total_amount_cells_expressing_gene_R,ident_R,mean_expr_R,var_expression_R,cell_type_R,variable_R
,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,ABCA7,APOA1,1,0.4572367,proteomics,Ligand,proteomics,69.10381,NA,proteomics_0_APOA1_P02647,⋯,neutrophil,Receptor,neutrophil,69.10381,NA,neutrophil_0_ABCA7,1.436823,0.8685307,neutrophil,neutrophil_0_ABCA7
2,ADIPOR1,ADIPOQ,1,0.7307022,proteomics,Ligand,proteomics,69.10381,NA,proteomics_0_ADIPOQ_Q15848,⋯,neutrophil,Receptor,neutrophil,69.10381,NA,neutrophil_0_ADIPOR1,2.384717,0.8461529,neutrophil,neutrophil_0_ADIPOR1


In [139]:
colnames(data)

[1] "Receptor"                            
 [2] "Ligand"                              
 [3] "amount_sources"                      
 [4] "weight"                              
 [5] "type"                                
 [6] "interaction_type"                    
 [7] "cluster"                             
 [8] "perc_cells_expressing_gene"          
 [9] "total_amount_cells_expressing_gene"  
[10] "ident"                               
[11] "mean_expr"                           
[12] "var_expression"                      
[13] "cell_type"                           
[14] "variable"                            
[15] "type_R"                              
[16] "interaction_type_R"                  
[17] "cluster_R"                           
[18] "perc_cells_expressing_gene_R"        
[19] "total_amount_cells_expressing_gene_R"
[20] "ident_R"                             
[21] "mean_expr_R"                         
[22] "var_expression_R"                    
[23] "cell_type_R"                         
[24] "variable_R"

In [140]:
nrow(data)

[1] 5826

In [141]:
unique(data$type)

[1] "proteomics"  "neutrophil"  "single_cell" "cytokine"

In [142]:
#### Remove same type links

In [143]:
data = data[data$cell_type != data$cell_type_R,]

In [144]:
nrow(data)

[1] 4921

In [145]:
unique(data$type)

[1] "proteomics"  "neutrophil"  "single_cell" "cytokine"

## Define expressed ligands and potential ligand - target combinations based on expressed receptors

In [146]:
### Define minimum amount of expression of receptor in receiving cell to be able to react to ligand

In [147]:
head(ligand_receptor_interactions,2)

,Ligand,Receptor,amount_sources,weight
,<chr>,<chr>,<int>,<dbl>
1,A2M,LRP1,1,0.4678977
2,A2M,TNFRSF14,1,0.6990671


In [148]:
head(cell_perc_cluster,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,X8_B.cell
2,AL627309.4,0.05406136,4,AL627309.4,X8_B.cell


In [149]:
### Set threshold to filter out receptors that are to low expressed
cell_perc_cluster_filtered =  cell_perc_cluster[((cell_perc_cluster$perc_cells > 30) & (cell_perc_cluster$total_amount_cells_expressing_gene > 600)) | ((cell_perc_cluster$perc_cells > 10) & (cell_perc_cluster$total_amount_cells_expressing_gene > 1200)),]


In [150]:
potential_ligand_target_types_comb = merge(ligand_receptor_interactions, cell_perc_cluster_filtered, by.x = 'Receptor', by.y = 'gene')

In [151]:
head(potential_ligand_target_types_comb,2)

,Receptor,Ligand,amount_sources,weight,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,cluster
,<chr>,<chr>,<int>,<dbl>,<chr>,<dbl>,<int>,<chr>
1,ABCA1,APOA1,1,0.8194814,ABCA19,15.74701,2303,X4_Monocytes...CD14
2,ABCA1,APOA1,1,0.8194814,ABCA17,16.46135,1844,X6_Monocytes...CD14


In [152]:
### Define all potential ligand receptor interactions based on expression
potential_ligand_target_types_comb = unique(potential_ligand_target_types_comb[,c('Ligand', 'cluster')] )

In [153]:
colnames(potential_ligand_target_types_comb) = c('Ligand', 'cluster_R')

In [154]:
head(potential_ligand_target_types_comb,2)

,Ligand,cluster_R
,<chr>,<chr>
1,APOA1,X4_Monocytes...CD14
2,APOA1,X6_Monocytes...CD14


In [155]:
unique(potential_ligand_target_types_comb$cluster_R)

[1] "X4_Monocytes...CD14"         "X6_Monocytes...CD14"        
 [3] "X2_T.cell.CD4"               "X1_T.cell.CD8"              
 [5] "X7_Monocytes...CD14"         "X0_T.cell.CD4"              
 [7] "X12_Monocytes...CD16_FCGR3A" "X3_NK"                      
 [9] "X9_Monocytes...CD16_FCGR3A"  "X8_B.cell"                  
[11] "X11_T.cell.CD4"              "X10_B.cell"                 
[13] "X5_T.cell.CD4"               "X13_Dendritic"

In [156]:
# Below: ligands where receptors are only expressed in very low amount of targets 
unique(potential_ligand_target_types_comb[,c('Ligand', 'cluster_R')]) %>% group_by(Ligand) %>% count() %>% filter(n == 1)


Ligand,n
<chr>,<int>
ARTN,1
CNTN2,1
DEFB4A,1
DEFB4B,1
DHH,1
FLT3LG,1
GPHA2,1
IHH,1
IL18,1


In [157]:
## get all expressed ligands per cluster

In [158]:
expressed_ligand_types = unique(data[,c('Ligand', 'cluster', 'mean_expr', 'type', 'variable', 'ident')])

# Ligand Target Data

## Calculate ligand - target correlations (lagged targets)

In [159]:
### expressed ligands per cluster
head(expressed_ligand_types,2)

,Ligand,cluster,mean_expr,type,variable,ident
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,APOA1,proteomics,8.733489,proteomics,proteomics_0_APOA1,proteomics_0_APOA1_P02647
2,ADIPOQ,proteomics,-1.154980,proteomics,proteomics_0_ADIPOQ,proteomics_0_ADIPOQ_Q15848


In [160]:
#### Determine expressed ligands in single-cell data
expressed_ligand_types$variable[expressed_ligand_types$type == 'single_cell'] = paste0(expressed_ligand_types$type[expressed_ligand_types$type == 'single_cell'], '_0_',expressed_ligand_types$cluster[expressed_ligand_types$type == 'single_cell'], '__', expressed_ligand_types$Ligand[expressed_ligand_types$type == 'single_cell'] )

In [161]:
expressed_ligands = unique(expressed_ligand_types$ident)

In [162]:
length(expressed_ligands)

[1] 343

In [163]:
### Transform data_long to matrix format

In [164]:
head(data_long,2)

,type,variable,X,sample_id,value,config,expression,ident,gene
,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>
1,clinical_data,CK,1557244,m1.1,7.467606,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,clinical_data_0_CK,CK
2,clinical_data,CK,1557245,m1.2,NA,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,clinical_data_0_CK,CK


In [165]:
expression_data = data_long %>% dcast(sample_id ~ ident, value.var = 'value')

In [166]:
rownames(expression_data) = expression_data$sample_id

In [167]:
expression_data$sample_id = NULL

In [168]:
head(expression_data,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_delta_ef_value,clinical_data_0_Troponin,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin__CCL11,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m1.1,7.467606,4.321928,3.137504,NA,0.75446,5.570766,9.747908,2.456806,6.335569,5.617357,⋯,2.201764,2.842767,2.610397,2.910977,2.461054,1.084743,0.7984776,0.4248882,1.451288,0.9409604
m1.2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [169]:
### Calculate all correlations between ligands and targets

In [170]:
head(expression_data,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_delta_ef_value,clinical_data_0_Troponin,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin__CCL11,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m1.1,7.467606,4.321928,3.137504,NA,0.75446,5.570766,9.747908,2.456806,6.335569,5.617357,⋯,2.201764,2.842767,2.610397,2.910977,2.461054,1.084743,0.7984776,0.4248882,1.451288,0.9409604
m1.2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [171]:
## Add lagged variables

In [172]:
expression_data_ligands = expression_data[,colnames(expression_data) %in% expressed_ligands]
expression_data_targets = expression_data[,!colnames(expression_data) %in% expressed_ligands]

In [173]:
expression_data_targets$sample = rownames(expression_data_targets)
expression_data_targets$tp = str_replace(expression_data_targets$sample, '.*\\.', '')

In [174]:
expression_data_targets$tp_lag = expression_data_targets$tp
expression_data_targets$tp_lag = str_replace(expression_data_targets$tp_lag, '1', '0')
expression_data_targets$tp_lag = str_replace(expression_data_targets$tp_lag, '2', '1')
expression_data_targets$tp_lag = str_replace(expression_data_targets$tp_lag, '3', '2')
expression_data_targets$tp_lag = str_replace(expression_data_targets$tp_lag, '4', '3')
expression_data_targets$sample = str_replace(expression_data_targets$sample, '\\..*', '')
expression_data_targets$sample_lag = paste0(expression_data_targets$sample, '.', expression_data_targets$tp_lag)

In [175]:
expression_data_targets_lagged1 = expression_data_targets
rownames(expression_data_targets_lagged1) = expression_data_targets_lagged1$sample_lag
expression_data_targets_lagged1$sample = NULL
expression_data_targets_lagged1$tp = NULL
expression_data_targets_lagged1$tp_lag = NULL
expression_data_targets_lagged1$sample_lag = NULL

In [176]:
head(expression_data_targets,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_delta_ef_value,clinical_data_0_Troponin,cytokine_0_Eotaxin__CCL11,cytokine_0_Eotaxin2__CCL24,cytokine_0_FLT3L__FLT3LG,cytokine_0_GROalpha__CXCL1,cytokine_0_IL12p40__IL12B,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX,sample,tp,tp_lag,sample_lag
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
m1.1,7.467606,4.321928,3.137504,NA,0.75446,5.617357,9.013713,3.944858,2.147307,3.502076,⋯,2.461054,1.084743,0.7984776,0.4248882,1.451288,0.9409604,m1,1,0,m1.0
m1.2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,m1,2,1,m1.1


In [177]:
head(expression_data_targets_lagged1,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_delta_ef_value,clinical_data_0_Troponin,cytokine_0_Eotaxin__CCL11,cytokine_0_Eotaxin2__CCL24,cytokine_0_FLT3L__FLT3LG,cytokine_0_GROalpha__CXCL1,cytokine_0_IL12p40__IL12B,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m1.0,7.467606,4.321928,3.137504,NA,0.75446,5.617357,9.013713,3.944858,2.147307,3.502076,⋯,2.201764,2.842767,2.610397,2.910977,2.461054,1.084743,0.7984776,0.4248882,1.451288,0.9409604
m1.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [178]:
### Merge lagged dataset by rownames
expression_data_lagged1 = merge(expression_data_ligands, expression_data_targets_lagged1,  by = 'row.names')

In [179]:
rownames(expression_data_lagged1 ) = expression_data_lagged1$Row.names
expression_data_lagged1$Row.names = NULL

In [180]:
head(expression_data_lagged1,2)

,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin3__CCL26,cytokine_0_FGF2__FGF2,cytokine_0_Fractalkine__CX3CL1,cytokine_0_GCSF__CSF3,cytokine_0_GMCSF__CSF2,cytokine_0_I309__CCL1,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
m1.1,5.570766,9.747908,2.456806,6.335569,1.883621,5.74873,5.874305,3.277985,0,1.090853,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
m1.2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [181]:
nrow(expression_data_lagged1)

[1] 84

In [182]:
### Calculate lagged correlations

In [183]:
ligand_target_correlations = cor(expression_data_lagged1[,colnames(expression_data_lagged1) %in% expressed_ligands], expression_data_lagged1[,!colnames(expression_data_lagged1) %in% expressed_ligands], method = 'spearman', use = 'pairwise.complete.obs') 

Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warning message in stats::cor(x, y, use, method):
“the standard deviation is zero”
Warn

In [184]:
head(ligand_target_correlations,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_delta_ef_value,clinical_data_0_Troponin,cytokine_0_Eotaxin__CCL11,cytokine_0_Eotaxin2__CCL24,cytokine_0_FLT3L__FLT3LG,cytokine_0_GROalpha__CXCL1,cytokine_0_IL12p40__IL12B,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
cytokine_0_BCA1__CXCL13,0.2267960,0.23290413,-0.03414252,-0.2322271,0.21535575,-0.1236559,0.01003276,-0.22891364,-0.3385113,-0.2668651,⋯,0.500991674,0.1973448,0.08517446,0.06217439,0.120236931,0.01181977,0.10062677,0.18792976,0.09434661,-0.02692659
cytokine_0_CTACK__CCL27,-0.1321796,-0.07370766,-0.21829420,0.0271015,0.07871666,-0.1298483,0.13897056,-0.06036676,0.1295516,0.3087579,⋯,-0.001533783,-0.1684650,-0.20541298,-0.42644065,0.008964222,-0.17581575,0.04926877,-0.05945203,0.16225078,-0.10117309


In [185]:
### Transform to long format

In [186]:
ligand_target_correlations = melt(ligand_target_correlations)

In [187]:
colnames(ligand_target_correlations) = c('Ligand', 'Target', 'correlation')

In [188]:
### Adjust and extract certain columns

In [189]:
ligand_target_correlations$Target_Gene = str_replace(ligand_target_correlations$Target, '.*_0_', '')

In [190]:
ligand_target_correlations$Target_Gene = str_replace(ligand_target_correlations$Target_Gene, '.*__', '')

In [191]:
ligand_target_correlations$Ligand_Gene = str_replace(ligand_target_correlations$Ligand, '.*_0_', '')

In [192]:
ligand_target_correlations$Ligand_Gene = str_replace(ligand_target_correlations$Ligand_Gene, '.*__', '')

In [193]:
ligand_target_correlations$Target_Type = str_replace(ligand_target_correlations$Target, '_0_.*', '')

In [194]:
ligand_target_correlations$Target_Type[ligand_target_correlations$Target_Type == 'single_cell']  = str_replace(ligand_target_correlations$Target[ligand_target_correlations$Target_Type == 'single_cell'] , '__.*', '')

In [195]:
###  Adapt target gene for proteomics
ligand_target_correlations$Target_Gene[ligand_target_correlations$Target_Type == 'proteomics'] = str_replace(ligand_target_correlations$Target_Gene[ligand_target_correlations$Target_Type == 'proteomics'], '_.*', '')
ligand_target_correlations$Ligand_Gene[ligand_target_correlations$Ligand_Type == 'proteomics'] = str_replace(ligand_target_correlations$Ligand_Gene[ligand_target_correlations$Ligand_Type == 'proteomics'], '_.*', '')

In [196]:
ligand_target_correlations$Ligand_Type = str_replace(ligand_target_correlations$Ligand, '_0_.*', '')

In [197]:
ligand_target_correlations$Ligand_Type[ligand_target_correlations$Ligand_Type == 'single_cell']  = str_replace(ligand_target_correlations$Ligand[ligand_target_correlations$Ligand_Type == 'single_cell'] , '__.*', '')

In [198]:
head(sort(unique(ligand_target_correlations$Ligand)))

[1] cytokine_0_BCA1__CXCL13    cytokine_0_CTACK__CCL27   
[3] cytokine_0_EGF__EGF        cytokine_0_ENA78__CXCL5   
[5] cytokine_0_Eotaxin3__CCL26 cytokine_0_FGF2__FGF2     
343 Levels: cytokine_0_BCA1__CXCL13 ... single_cell_0_X9_Monocytes...CD16_FCGR3A__TNFSF13B

In [199]:
unique(ligand_target_correlations$Target_Type)

[1] "clinical_data"                            
 [2] "cytokine"                                 
 [3] "neutrophil"                               
 [4] "proteomics"                               
 [5] "single_cell_0_X0_T.cell.CD4"              
 [6] "single_cell_0_X1_T.cell.CD8"              
 [7] "single_cell_0_X10_B.cell"                 
 [8] "single_cell_0_X11_T.cell.CD4"             
 [9] "single_cell_0_X12_Monocytes...CD16_FCGR3A"
[10] "single_cell_0_X13_Dendritic"              
[11] "single_cell_0_X2_T.cell.CD4"              
[12] "single_cell_0_X3_NK"                      
[13] "single_cell_0_X4_Monocytes...CD14"        
[14] "single_cell_0_X5_T.cell.CD4"              
[15] "single_cell_0_X6_Monocytes...CD14"        
[16] "single_cell_0_X7_Monocytes...CD14"        
[17] "single_cell_0_X8_B.cell"                  
[18] "single_cell_0_X9_Monocytes...CD16_FCGR3A"

In [200]:
ligand_target_correlations$cluster = str_replace(ligand_target_correlations$Target_Type, 'single_cell_0_', '')

In [201]:
unique(ligand_target_correlations$cluster)

[1] "clinical_data"               "cytokine"                   
 [3] "neutrophil"                  "proteomics"                 
 [5] "X0_T.cell.CD4"               "X1_T.cell.CD8"              
 [7] "X10_B.cell"                  "X11_T.cell.CD4"             
 [9] "X12_Monocytes...CD16_FCGR3A" "X13_Dendritic"              
[11] "X2_T.cell.CD4"               "X3_NK"                      
[13] "X4_Monocytes...CD14"         "X5_T.cell.CD4"              
[15] "X6_Monocytes...CD14"         "X7_Monocytes...CD14"        
[17] "X8_B.cell"                   "X9_Monocytes...CD16_FCGR3A"

In [202]:
head(ligand_target_correlations,2)

,Ligand,Target,correlation,Target_Gene,Ligand_Gene,Target_Type,Ligand_Type,cluster
,<fct>,<fct>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,cytokine_0_BCA1__CXCL13,clinical_data_0_CK,0.2267960,CK,CXCL13,clinical_data,cytokine,clinical_data
2,cytokine_0_CTACK__CCL27,clinical_data_0_CK,-0.1321796,CK,CCL27,clinical_data,cytokine,clinical_data


## Filter on ligands with matching receptor in target

In [203]:
head(potential_ligand_target_types_comb,2) # dataset with available receptor for ligands in target types

,Ligand,cluster_R
,<chr>,<chr>
1,APOA1,X4_Monocytes...CD14
2,APOA1,X6_Monocytes...CD14


In [204]:
nrow(ligand_target_correlations)

[1] 4438077

In [205]:
filter_matched_receptor = TRUE

In [206]:
if(filter_matched_receptor == TRUE){
    ### seperately add clinical, proteomics as target (as here no filtering based on expression is applied)
    no_sc = ligand_target_correlations[ligand_target_correlations$Target_Type %in%  c('clinical_data', 'proteomics', 'neutrophil'),]
    ### filter the ligands
    ligand_target_correlations = merge(ligand_target_correlations, potential_ligand_target_types_comb[,c('Ligand', 'cluster_R')], by.x = c('Ligand_Gene', 'cluster'), by.y = c('Ligand', 'cluster_R'))
    ### add other omics data
    ligand_target_correlations = rbind(ligand_target_correlations, no_sc)
    
}

In [207]:
nrow(ligand_target_correlations)

[1] 3510462

## Add regulatory potential

In [208]:
## add ligand target regulatory potential score from nichenet to the data

In [209]:
head(ligand_target_matrix_long,2)# regulatory potential scores from nichenet

,target,ligand,regulatory_potential
,<fct>,<fct>,<dbl>
1,A1BG,CXCL1,0.0003534343
2,A1BG-AS1,CXCL1,0.0001650894


In [210]:
### Adjust gene names for clinical data
ligand_target_correlations$Target_Gene[ligand_target_correlations$Target == 'clinical_data_0_CK'] = 'CKM'
ligand_target_correlations$Target_Gene[ligand_target_correlations$Target == 'clinical_data_0_CK_MB'] = 'CKB'
ligand_target_correlations$Target_Gene[ligand_target_correlations$Target == 'clinical_data_0_Troponin'] = 'TNNT2'

In [211]:
### Merge regulatory potential scores

In [212]:
ligand_target_correlations = merge(ligand_target_correlations, ligand_target_matrix_long, by.x = c('Ligand_Gene', 'Target_Gene'), by.y = c('ligand', 'target'))

In [213]:
nrow(ligand_target_correlations)

[1] 3302756

In [214]:
## scale the regulatory potential score based on max value

In [215]:
ligand_target_correlations$regulatory_potential_scaled = ligand_target_correlations$regulatory_potential/ max(ligand_target_correlations$regulatory_potential)

## Adjustments to ligand-target correlations

In [216]:
### Remove cytokines that have been calculated as targets (because not included in ligand list above)

In [217]:
nrow(ligand_target_correlations)

[1] 3302756

In [218]:
ligand_target_correlations = ligand_target_correlations[ligand_target_correlations$Target_Type != 'cytokine',]

In [219]:
nrow(ligand_target_correlations)

[1] 3302756

In [220]:
### Only look at between cell-type communications --> remove same cell-type Links

In [221]:
ligand_target_correlations$Target_Cell_Type = str_replace(ligand_target_correlations$Target_Type, '.*_X\\d_|.*_X\\d\\d_','')

In [222]:
ligand_target_correlations$Ligand_Cell_Type = str_replace(ligand_target_correlations$Ligand_Type, '.*_X\\d_|.*_X\\d\\d_','')

In [223]:
ligand_target_correlations = ligand_target_correlations[ligand_target_correlations$Target_Cell_Type != ligand_target_correlations$Ligand_Cell_Type,]

In [224]:
nrow(ligand_target_correlations)

[1] 2709661

## Save the ligand target data

In [225]:
name

[1] "V_FINAL"

In [226]:
filter_samples

[1] "acs"

In [227]:
write.table(ligand_target_correlations,gzfile(paste0(result_path, '/F-Analysis/' ,'F4_Ligand_Target_Data', name, filter_samples,'.gz')))